In [ ]:
## REAL TIME PLOTTING CODE - GOAL: SEE WHAT IS HAPPENING ON THE ARRAY RIGHT NOW


## STEP 1: INITIALIZE NON-CHANGING ASPECTS

#General dependencies (some of these may not be used - check at the end)
import numpy as np
import scipy.io as sio
from os.path import dirname, join as pjoin
import time
import matplotlib.pyplot as plt
%matplotlib tk
import pandas as pd
from matplotlib.animation import FuncAnimation
import glob
import os

# Setting Params for plotting & troubleshoot
np.set_printoptions(threshold=np.inf)
plt.rcParams['animation.html']='jshtml'

In [ ]:
## STEP 2: INITIALIZE CHANGING VALUES

# Initialize the Path and Data Name
path = r'/Shakedown/Users/vision/Desktop/Xilinx'

date_piece = '2021-08-12-0'
run = 0
file_num = 0

chan_idx = 587 #The electrode of interest (1 to 1024)
time_win = 20 #Number of samples of interest (i.e. 3ms/0.05 ms per sample = 60 samples)
real_time_win = 300

# Convert Path & Data for Search 
datarun = 'data'+str(run).zfill(3)
folder = datarun+'_'+str(file_num)
file_type = '/*mat'
latest_file_pre = ' '

# Initialize Figure
plt.close('all')
fig, ax = plt.subplots(2,1,figsize=(10,10),gridspec_kw={'height_ratios':[2,1]})
fig.show()
plt.style.use('fivethirtyeight')
plt.grid(True)
plt.subplots_adjust(left=0.345,right=0.95,bottom=0.07,top=0.92)
ax[0].set_title('Whole Array Activity')

# Reads for the largest file number of the specified data run
def func(p):
    return int(((p.split('.')[0]).split('_'))[-1])

## STEP 3: PRE-LOOP HOLDS (i.e. can run this before or after data is sent)

# If the file does not yet exist, stay here
while not(os.path.isdir(pjoin(date_piece, datarun))):
    continue

# If the file exists, but hasn't yet been filled, hold here
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    if len(data_dir)>0:
        break
        
# STEP 4: ALWAYS BE CHECKING FOR DATA

# Continuously Check for New Data
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)

    # In case multiple files coming in at once, hold 100ms (not likely)
    #time.sleep(0.1)
    
    # Choose the file by the largest file number
    latest_file = max(data_dir, key=func)
    
    # If the latest file is the same as the previously plotted file, hold
    if latest_file == latest_file_pre:
        continue
    
    print(latest_file)
    
    # Given the new file, update the previous file check for next time
    latest_file_pre = latest_file
    
    # In the off chance the file has been written, but not saved by 
    # the TCP socket yet, pause
    time.sleep(0.1)
    
    ## STEP 5: PROCESS DATA 
    
    # Load the newest data (NOTE: Change to vect for full array test)
    mat_contents = sio.loadmat(latest_file)
    data = mat_contents['dummy_data_ch586_hard_gnd'][0][:]
    
    # Convert the data
    data_real = np.zeros((32,32,len(data)-2))  #Data should be a power/divisible by 2^N where N= number of channels active in the recording buffer. Note Divide by 2 because of the double count and -1 because the first sample (w/o double count) is garbo
    cnt_real = np.zeros((32,32,len(data)-2))
    #counter = np.zeros((1,1,len(data)-2)) - Uncomment once data w/ counter
    
    # Initialize Variables for the Loop
    cnt_pre = 0
    N = 0 # Sample times
    chan_index_pre = 1025 #Make sure it is not a true channel
    #counter_pre = 0 - Uncomment once data w/ counter
    
    for i in range(2,len(data)-1): 
        # Convert bit number into binary
        word = (np.binary_repr(data[i],32))
        
        # Break that binary into it's respective pieces and convert to bit number
        cnt = int(word[12:14],2)
        col = int(word[27:32],2)
        row = int(word[22:27],2)
        chan_index = row*32 + col
        #count_all = int(word[1:12],2) - Uncomment once data w/ counter
        
        # Only record the unique non-double count sample
        if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):
            
            # Sample time only changes when cnt changes
            if cnt != cnt_pre:
                N += 1
                
            # On the occurance the first cnt is not 0, make sure sample time is 0
            if i == 2:
                N = 0
                    
            # Uncomment this once data with counter comes in
            #if counter_pre != count_all:
                #count[0,0,counter_pre] = count_all
                
            # Update variables
            cnt_pre = cnt
            chan_index_pre = chan_index
            
            # Record pertinent data
            data_real[row][col][N] = int(word[14:22],2)
            cnt_real[row][col][N] = cnt
            
    # Truncate data to be the number of samples       
    num_samples = N
    data_real = data_real[:,:,:N]
    cnt_real = cnt_real[:,:,:N]
    #total_time = count_all*0.05 - Uncomment once data w/ counter
    
    
    ## STEP 6: PREP FOR PLOTTING 

    # Initialize variables
    max_cnt = 20 #Saturation point for number of samples
    max_dot = 600 #Saturation point for dot size
    sample_win = int(time_win/0.05)
    
    # Initialize Electrode Indices
    chan_num = int(chan_idx-1)
    chan_row = int(chan_num / 32)
    chan_col = chan_num - (chan_row * 32)

    # Determine the Time of Each Sample (NOTE: Once counter is in place, this needs adjusted so any non-collision free samples appear void of values)
    total_time = num_samples*0.05 #Sampling rate 1/0.05 ms 
    times = np.linspace(0,total_time,num_samples)
    
    # Initialize Array Indices
    base = np.arange(1,33)
    rows = [ele for ele in base for i in range(32)]
    temp = [1 for i in range(32)]
    cols = [ele * tele for tele in temp for ele in base]

    # Initialize Variables for the Loop (size and colors will be returned) 
    size = np.zeros((32,32,0)) # For each dot
    num_sam = np.zeros((32,32,1))
    chan_sam = np.zeros((32,32,0))
    chan_avg = np.zeros((32,32,0))
    colors = np.zeros((32,32,0)) # For each dot
    avg_val = np.zeros((32,32,1))
    max_num_temp = np.zeros((2,1))
    
    # Start with the Last Bit of Information
    if total_time > real_time_win:
        i = num_samples - int(real_time_win/0.05)
    elif total_time <= real_time_win:
        i = 0
    
    # Calculate plotting values over entire given sample space
    while i < num_samples:
    
        # Determine the non-zero collection of samples in time window and max number of samples in time window
        num_sam[:,:,0] = np.count_nonzero(data_real[:,:,i:i+sample_win],axis=2)
        chan_sam = np.append(chan_sam,num_sam,axis=2)
        max_num_temp[0,0] = np.max(num_sam[:,:,0])
        max_num_temp[1,0] = np.argmax(num_sam[:,:,0])
    
        # Determine average value of samples in time window and max value of samples in time window
        num_sam[num_sam==0] = np.nan
        avg_val[:,:,0] = np.sum(data_real[:,:,i:i+sample_win],axis=2)/num_sam[:,:,0]
        chan_avg = np.append(chan_avg,avg_val,axis=2)
    
        # Record the average values to represent colors for each time window
        # NOTE: Using append b/c it does not require exact calculations. May be a faster version of append (i.e. concatenate)
        colors = np.append(colors,avg_val,axis=2)
    
        num_sam = np.nan_to_num(num_sam,nan=0)
        # Scale size according to the max value unless that value exceeds the saturation point. Squaring in the scale to emphasize differences
        if max_num_temp[0,0] < max_cnt:
            scale = (max_dot - 10) / (max_num_temp[0,0]**2)
            size = np.append(size, num_sam*scale + 10, axis = 2)
        elif max_num_temp[0,0] >= max_cnt:
            scale = (max_dot - 10) / (max_cnt**2)
            size = np.append(size, num_sam*scale + 10, axis = 2)
    
        # Update i by the time_window
        i += sample_win
        
    chan_max_sam = np.sum(chan_sam,axis=2)
    chan_mark1 = [np.max(chan_max_sam),np.argmax(chan_max_sam)]
    chan_avg_val = np.sum(chan_avg,axis=2)/len(chan_avg[0][0])
    chan_avg_val = np.nan_to_num(chan_avg_val,nan=0)
    chan_mark2 = [np.max(chan_avg_val),np.argmax(chan_avg_val)]
        
    ## STEP 7: PLOT THAT DATA
    # Note: This can be applied at the end of the last cell, but Jupyter
    # Notebooks gets angry about this even though the plotting goes on as normal

    # NOTE: This needs updated to plot the most recent of real-time data
    # and to hold on the last figure of true data while we are waiting for 
    # another file
    
    # Initialize every time
    x1 = rows
    y1 = cols
    x2 = np.empty(0)
    y2 = np.empty(0)
    n = 0
    
    # Initialize plot (speeds it up but removes previous buffer info - this happens anyway until we use counter instead of count for timing)
    ax[1].clear()
    ax[1].set_title('Electrode # ' + str(chan_idx))
    ax[1].set(xlabel = 'Time (ms)', ylabel = 'Voltage')
    textstr2 = 'Time Window = %.2f ms' %(time_win)
    textstr3 = 'GUI CONTROLS\n\n'
    textstr4 = 'Pan/Zoom: Toggle [p], Click [l/r], Drag\nSave: [s]\nConstrain Pan/Zoom: Hold [x/y]\nPreserve Aspect Ratio: Hold [ctrl]\nReset: [r]'
    ax[1].text(-0.55,0,textstr3+textstr4,transform=ax[1].transAxes,fontsize=9,bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})

    
    #Start with the last bit of information but plot all previous
    if total_time > real_time_win:
        i = num_samples - int(real_time_win/0.05)
        x2 = np.append(x2,times[0:num_samples-int(real_time_win/0.05)-1])
        y2 = np.append(y2,data_real[chan_row,chan_col,0:num_samples-int(real_time_win/0.05)-1])
        ax[1].plot(x2,y2,color='r')
        fig.canvas.draw()
        fig.canvas.flush_events()
    elif total_time <= real_time_win:
        i = 0

    # Plot for the entire length of incoming data
    tic = time.time()
    while num_samples > i:
    
        # Clear Scatter Plot
        ax[0].clear()
    
        # For the electrode plot, append the new data to be plotted
        x2 = np.append(x2,times[i:i+sample_win])
        y2 = np.append(y2, data_real[chan_row, chan_col, i:i+sample_win])
    
        if n >= len(size[0][0]):
            n = len(size[0][0])-1
    
        # For the scatter plot, update the size and colors
        ax[0].scatter(x1,y1,size[:,:,n],c=colors[:,:,n],cmap='jet')
        ax[0].set_title('Whole Array Activity')
        ax[0].set_xlim(left=0,right=31)
        ax[0].set_ylim(bottom=0,top=31)
    
        # For the electrode plot, set a new xlimit
        ax[1].plot(x2,y2,color='r')
        ax[1].set_xlim(left=max(0,int(times[i])-5),right=(int(times[i])+time_win+1))
    
        # Text Box of the Electrode with Maximum Amplitude
        # NOTE: For a quick speed up, can move this in reference with ax[1] which doesn't get cleared before the loop
        textstr = 'Max Avg Amp\nElec = {0}\nAmp = {1:3.2f}\n\nMax Num Samp\nElec = {2}\nNum={3:.0f}'.format(chan_mark2[1],chan_mark2[0],chan_mark1[1],chan_mark1[0])
        ax[0].text(-0.4,0.25,textstr,transform=ax[0].transAxes,fontsize=14)
        ax[0].text(-0.55, 1.05, textstr2,transform=ax[0].transAxes,fontsize=14)

        # This step is fast but most time consuming - may replace with fig.canvas.update()
        fig.canvas.draw()
    
        # Flush events (equivalent to time.sleep or plt.pause but faster)
        if len(size[0][0]) != n+1:
            fig.canvas.flush_events()
    
        # Use time.sleep if you want to plot really slowly you weirdo
        #time.sleep(1)
    
        # Update the variables
        i += sample_win
        n += 1
        
        # Once at the end, go back to the beginning
        if i >= num_samples:
            break
            
    toc = time.time()
    print('Time to Plot New Data ' + str(toc-tic))
        
        
# NOTE: Put in interrupt on updating plot to look for new data coming in
# Put in a Pause Button




